In [1]:
import numpy as np
import pickle
from tqdm import tqdm
from irt import *
from utils import *

random_state = 42

model_list = [
    'microsoft/Phi-3-medium-4k-instruct',
    'internlm/internlm2_5-7b-chat',
    'microsoft/Phi-3-small-128k-instruct',
    '01-ai/Yi-1.5-9B-Chat',
    'MaziyarPanahi/Llama-3-8B-Instruct-v0.8',
    'Qwen/Qwen2-7B-Instruct',
    'NousResearch/Hermes-2-Theta-Llama-3-8B',
    'vicgalle/Roleplay-Llama-3-8B',
    'Qwen/Qwen2-7B',
    'NousResearch/Nous-Hermes-2-SOLAR-10.7B',
    'UCLA-AGI/Llama-3-Instruct-8B-SPPO-Iter3',
    '01-ai/Yi-1.5-9B-Chat-16K',
    'refuelai/Llama-3-Refueled',
    'openchat/openchat-3.6-8b-20240522',
    'openchat/openchat-3.5-0106',
    'openchat/openchat-3.5-1210',
    '01-ai/Yi-1.5-6B-Chat',
    'mlabonne/NeuralDaredevil-8B-abliterated',
    '01-ai/Yi-1.5-9B',
    'NousResearch/Hermes-2-Pro-Mistral-7B',
    'NousResearch/Hermes-2-Pro-Llama-3-8B',
    'openchat/openchat_3.5',
    'Intel/neural-chat-7b-v3-2',
    'teknium/OpenHermes-2-Mistral-7B',
    'teknium/OpenHermes-2.5-Mistral-7B',
    'NousResearch/Nous-Hermes-2-Mistral-7B-DPO',
    'Intel/neural-chat-7b-v3-1',
    'berkeley-nest/Starling-LM-7B-alpha',
    'Intel/neural-chat-7b-v3-3',
    'upstage/SOLAR-10.7B-Instruct-v1.0',
    # more
    'RLHFlow/LLaMA3-iterative-DPO-final',
    '01-ai/Yi-1.5-9B-32K',
    'mistralai/Mistral-7B-Instruct-v0.3',
    'HuggingFaceH4/zephyr-7b-alpha',
    'mistralai/Mistral-7B-Instruct-v0.2'
    'cognitivecomputations/dolphin-2.9-llama3-8b',
    'meta-llama/Llama-2-70b-hf',
    'microsoft/Orca-2-13b',
    'gradientai/Llama-3-8B-Instruct-Gradient-1048k',
    'THUDM/glm-4-9b',
    'Intel/neural-chat-7b-v3',
    'HuggingFaceH4/zephyr-7b-beta',
    'Open-Orca/Mistral-7B-OpenOrca',
    '01-ai/Yi-9B',
    '01-ai/Yi-9B-200K',
    'Deci/DeciLM-7B-instruct',
    'google/gemma-1.1-7b-it',
    'upstage/SOLAR-10.7B-v1.0',
    'ibm/merlinite-7b',
    'LLM360/CrystalChat',
    '01-ai/Yi-1.5-6B',
    'stabilityai/stablelm-2-12b-chat',
    'CohereForAI/aya-23-8B',
    'HuggingFaceH4/zephyr-7b-gemma-v0.1',
    'NousResearch/Yarn-Solar-10b-32k',
    'microsoft/phi-2',
    'google/gemma-7b',
    'Qwen/Qwen1.5-7B',
    'WizardLMTeam/WizardLM-13B-V1.2',
    'TencentARC/LLaMA-Pro-8B-Instruct',
    'NousResearch/Yarn-Solar-10b-64k',
    'Deci/DeciLM-7B',
    'mlabonne/OrpoLlama-3-8B',
    'deepseek-ai/deepseek-llm-7b-chat',
    'mistralai/Mistral-7B-v0.1',
    'teknium/CollectiveCognition-v1.1-Mistral-7B',
    'TencentARC/Mistral_Pro_8B_v0.1',
    'mistralai/Mistral-7B-v0.3',
    'microsoft/Orca-2-7b',
    'mistral-community/Mistral-7B-v0.2',
    '01-ai/Yi-6B-Chat',
    'Qwen/Qwen2-1.5B-Instruct',
    'stabilityai/stablelm-2-12b',
    'openchat/openchat_v3.2',
    'tiiuae/falcon-11B',
    '01-ai/Yi-6B',
    'mistralai/Mistral-7B-Instruct-v0.1',
    'NousResearch/Yarn-Mistral-7b-64k',
]

log_dir = '/data/amos_zhu/lm-evaluation-harness/logs'

In [3]:
# read pkl
with open(f'data/anchor.pickle', 'rb') as f:
    data = pickle.load(f)

print(data.keys())


dict_keys(['anchor_points', 'anchor_weights'])


In [2]:
scenarios = {'wmdp': ['wmdp_cyber', 'wmdp_bio']}

"\n{'correctness': array[q_num][model_num])\n}\n"

In [22]:
import json

def extract_data(model_dir):
    log_names = os.listdir(model_dir)
    wmdp_bio_log = None
    wmdp_cyber_log = None
    results_log = None
    for log_name in log_names:
        if 'wmdp_bio' in log_name:
            wmdp_bio_log_dir = os.path.join(model_dir, log_name)
            with open(wmdp_bio_log_dir, 'r') as f:
                wmdp_bio_log = []
                for line in f:
                    json_data = json.loads(line.strip())
                    wmdp_bio_log.append(json_data)
            
                
        elif 'wmdp_cyber' in log_name:
            wmdp_cyber_log_dir = os.path.join(model_dir, log_name)
            with open(wmdp_cyber_log_dir, 'r') as f:
                wmdp_cyber_log = []
                for line in f:
                    json_data = json.loads(line.strip())
                    wmdp_cyber_log.append(json_data)
        elif 'results' in log_name:
            results_log_dir = os.path.join(model_dir, log_name)
            with open(results_log_dir, 'r') as f:
                results_log = json.load(f)
    
    return wmdp_bio_log, wmdp_cyber_log, results_log
            
            


wmdp_bio_logs = []
wmdp_cyber_logs = []
results_logs = []
model_names = []

import os
for model_name in model_list:
    model_name = model_name.split('/')
    org_dir = os.path.join(log_dir, model_name[0], model_name[1])
    full_model_name = '__'.join(model_name)
    model_dir = os.path.join(org_dir, full_model_name)

    if not os.path.exists(model_dir):
        # print(f'{full_model_name} not found')
        # failed_model_num += 1
        continue

    wmdp_bio_log, wmdp_cyber_log, results_log = extract_data(model_dir)
    wmdp_bio_logs.append(wmdp_bio_log)
    wmdp_cyber_logs.append(wmdp_cyber_log)
    results_logs.append(results_log)
    model_names.append(full_model_name.replace('__', '/'))

# print(f'{failed_model_num} out of {total_model_num} models not found')

In [33]:
def process_wmdp_logs(wmdp_logs):
    '''
    {'correctness': array[q_num][model_num])
    }
    '''
    num_models = len(wmdp_logs)
    num_questions = len(wmdp_logs[0])
    print(f'num_models: {num_models}, num_questions: {num_questions}')
    correctness = np.zeros((num_questions, num_models))
    for i, wmdp_log in enumerate(wmdp_logs):
        wmdp_log.sort(key=lambda x: x['doc_id'])
        for j, log in enumerate(wmdp_log):
            q_index = int(log['doc_id'])
            model_index = i
            correctness[q_index, model_index] = log['acc']
    return {'correctness': correctness}

def process_logs(wmdp_bio_logs,
                 wmdp_cyber_logs,
                 results_logs,
                 model_names):
    data = {'models': [], 'data': {}}  

    data['models'] = model_names

    bio_matrix = process_wmdp_logs(wmdp_bio_logs)
    cyber_matrix = process_wmdp_logs(wmdp_cyber_logs)
    data['data']['wmdp_bio'] = bio_matrix
    data['data']['wmdp_cyber'] = cyber_matrix

    # save to pickle
    output_dir = '/data/amos_zhu/tinyBenchmarks/tutorials'
    output_dir = os.path.join(output_dir, f'wmdp_data.pkl')
    with open(output_dir, 'wb') as f:
        pickle.dump(data, f)
    print(f'Saved to {output_dir}')

process_logs(wmdp_bio_logs, wmdp_cyber_logs, results_logs, model_names)

num_models: 64, num_questions: 1273
num_models: 64, num_questions: 1987
Saved to /data/amos_zhu/tinyBenchmarks/tutorials/wmdp_data.pkl


In [34]:
# try load
output_dir = '/data/amos_zhu/tinyBenchmarks/tutorials'
output_dir = os.path.join(output_dir, f'wmdp_data.pkl')
with open(output_dir, 'rb') as f:
    data = pickle.load(f)
    print(data['models'])
    print(data['data']['wmdp_bio']['correctness'].shape)
    print(data['data']['wmdp_cyber']['correctness'].shape)

['microsoft/Phi-3-medium-4k-instruct', 'internlm/internlm2_5-7b-chat', '01-ai/Yi-1.5-9B-Chat', 'MaziyarPanahi/Llama-3-8B-Instruct-v0.8', 'Qwen/Qwen2-7B-Instruct', 'NousResearch/Hermes-2-Theta-Llama-3-8B', 'vicgalle/Roleplay-Llama-3-8B', 'Qwen/Qwen2-7B', 'NousResearch/Nous-Hermes-2-SOLAR-10.7B', 'UCLA-AGI/Llama-3-Instruct-8B-SPPO-Iter3', '01-ai/Yi-1.5-9B-Chat-16K', 'refuelai/Llama-3-Refueled', 'openchat/openchat-3.6-8b-20240522', 'openchat/openchat-3.5-0106', 'openchat/openchat-3.5-1210', '01-ai/Yi-1.5-6B-Chat', 'mlabonne/NeuralDaredevil-8B-abliterated', '01-ai/Yi-1.5-9B', 'NousResearch/Hermes-2-Pro-Mistral-7B', 'NousResearch/Hermes-2-Pro-Llama-3-8B', 'openchat/openchat_3.5', 'Intel/neural-chat-7b-v3-2', 'teknium/OpenHermes-2-Mistral-7B', 'teknium/OpenHermes-2.5-Mistral-7B', 'NousResearch/Nous-Hermes-2-Mistral-7B-DPO', 'Intel/neural-chat-7b-v3-1', 'berkeley-nest/Starling-LM-7B-alpha', 'Intel/neural-chat-7b-v3-3', 'upstage/SOLAR-10.7B-Instruct-v1.0', 'RLHFlow/LLaMA3-iterative-DPO-final',